<a href="https://colab.research.google.com/github/JohnTan38/LangChain-and-LLM/blob/master/YT_ChatGPT_API_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install openai langchain==0.0.99rc0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.2/348.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 KB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 9.1 MB/s eta 0:0

## Basics with OpenAI API

In [12]:
import os
import openai

openai.api_key =''
os.environ['OPENAI_API_KEY'] = ''

In [5]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]
)

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-6pakUtAnnDzKpBkh2QcxLAp1ymTtR at 0x7fdae98f9400> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I am a virtual assistant, equipped with AI technology to assist you with various tasks and answer your questions as best as I can. How may I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1677754010,
  "id": "chatcmpl-6pakUtAnnDzKpBkh2QcxLAp1ymTtR",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 35,
    "prompt_tokens": 26,
    "total_tokens": 61
  }
}

### Chat Markup Language - token system

```markdown
<|im_start|>system
You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.
Knowledge cutoff: 2021-09-01
Current date: 2023-03-01<|im_end|>
<|im_start|>user
How are you<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
How are you now?<|im_end|>
```

```
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```

In [6]:
messages=[
        {"role": "system", "content": "You are a helpful assistant named Kate."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]

In [ ]:
conversation_total_tokens = 0

while True:
    message = input("Human: ")
    if message=='exit':
        print(f"{conversation_total_tokens} tokens used in total in this conversation")
        break
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    
    reply = response.choices[0].message.content
    total_tokens = response.usage['total_tokens']
    conversation_total_tokens += total_tokens
    print(f"ChatGPT: {reply} \n {total_tokens} tokens used")
    messages.append({"role": "assistant", "content": reply})

## ChatGPT with LangChain

In [8]:
!pip show langchain

Name: langchain
Version: 0.0.99rc0
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: aiohttp, aleph-alpha-client, dataclasses-json, deeplake, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [9]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, OpenAIChat

In [10]:
prefix_messages = [{"role": "system", "content": "You are a helpful history professor named Kate."}]


In [13]:
## old way
# llm = OpenAI(model_name="text-davinci-003",
#              temperature=0, )

## New way
llm = OpenAIChat(model_name='gpt-3.5-turbo', 
             temperature=0, 
             prefix_messages=prefix_messages,
             max_tokens = 256)

In [14]:

template = """Take the following question: {user_input}

Answer it in an informative and intersting but conscise way for someone who is new to this topic."""

prompt = PromptTemplate(template=template, 
                        input_variables=["user_input"])


In [15]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "When was Marcus Aurelius the emperor of Rome?"

llm_chain.run(user_input)

'Marcus Aurelius was the emperor of Rome from 161 to 180 AD. He was known for his philosophical writings, particularly his book "Meditations," which is still studied today. During his reign, he faced numerous challenges, including wars with Germanic tribes and a devastating plague. Despite these difficulties, he is remembered as one of Rome\'s greatest emperors, known for his wisdom, integrity, and dedication to duty.'

In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "Who was Marcus Aurelius married to?"

llm_chain.run(user_input)

'Marcus Aurelius was married to Faustina the Younger, who was the daughter of Antoninus Pius, the emperor before Marcus Aurelius. Faustina was known for her beauty and intelligence, and she played an important role in the political and social life of Rome. She accompanied her husband on his military campaigns and was revered by the people of Rome for her charitable works. Despite rumors of infidelity, Marcus Aurelius remained devoted to Faustina throughout their marriage, and he even deified her after her death.'